In [1]:
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark import SparkConf, SparkContext, StorageLevel
from pyspark.sql.session import SparkSession

import numpy as np

from pprint import pprint

In [2]:
config_cluster = SparkConf().setMaster("local").setAppName("bd_pi3")
sc = SparkContext(conf=config_cluster)
spark = SparkSession(sc)

In [3]:
path = "/home/victor/Documentos/github/bd2_pi3/data/20-newsgroups/*.txt"

re = sc.newAPIHadoopFile(
    path,
    "org.apache.hadoop.mapreduce.lib.input.TextInputFormat",
    "org.apache.hadoop.io.LongWritable",
    "org.apache.hadoop.io.Text",
    conf={"textinputformat.record.delimiter": "\nNewsgroup"},
)

In [4]:
lenWord = 5
"""
: sci.space\nDocument_id: 62474\nFrom: lazio@astrosun.tn.cornell.edu (T. Joseph Lazio)\nSubject: Re: Space Marketing would be wonderfull.\n\n>>>>> On Sun, 16 May 1993 14:31:20 GMT, fcrary@ucsu.Colorado.EDU (Frank Crary) said:\nfc> Modern, \nfc> ground-based, visible light astronomy (what these proposed\nfc> orbiting billboards would upset) is already a dying field: The\nfc> opacity and distortions caused by the atmosphere itself have\nfc> driven most of the field to use radio, far infrared or space-based\nfc> telescopes. \n\n Here\'s one radio astronomer quite concerned about \n radio-frequency interference from portable telephones, etc.\n\n\n--\n                         | e-mail: lazio@astrosun.tn.cornell.edu\n   T. Joseph Lazio       | phone:  (607) 255-6420\n                         | ICBM:   42 deg. 20\' 08" N  76 deg. 28\' 48" W\nCornell knows I exist?!? |       STOP RAPE\n
"""

'\n: sci.space\nDocument_id: 62474\nFrom: lazio@astrosun.tn.cornell.edu (T. Joseph Lazio)\nSubject: Re: Space Marketing would be wonderfull.\n\n>>>>> On Sun, 16 May 1993 14:31:20 GMT, fcrary@ucsu.Colorado.EDU (Frank Crary) said:\nfc> Modern, \nfc> ground-based, visible light astronomy (what these proposed\nfc> orbiting billboards would upset) is already a dying field: The\nfc> opacity and distortions caused by the atmosphere itself have\nfc> driven most of the field to use radio, far infrared or space-based\nfc> telescopes. \n\n Here\'s one radio astronomer quite concerned about \n radio-frequency interference from portable telephones, etc.\n\n\n--\n                         | e-mail: lazio@astrosun.tn.cornell.edu\n   T. Joseph Lazio       | phone:  (607) 255-6420\n                         | ICBM:   42 deg. 20\' 08" N  76 deg. 28\' 48" W\nCornell knows I exist?!? |       STOP RAPE\n\n'

In [5]:
def getIdDocument(line):
    line = line.split('Document_id:')
    if len(line) > 1:
        line = line[1]
    else:
        return None
    line = line.split('\n')[0]
    line = line.split(' ')
    if len(line) > 1:
        line = line[1]
    return line

In [6]:
def removeChar(line):
    charsToRemove = list(np.arange(65))
    charsToRemove.extend(np.arange(91,97))
    charsToRemove.extend(np.arange(123,255))

    for char in charsToRemove:
        try:
            line = line.replace(chr(char),' ')
        except:
            pass

    return line

In [7]:
def getFeatures((num, line)):
    line = line.split('Subject:')
    setWords = set() # set of words of document
    
    idDoc = getIdDocument(line[0])
    if idDoc == None:
        return None

    if len(line) > 1:
        line = line[1]
    else:
        return None
    
    line = removeChar(line)
    for word in line.split(' '):
        if len(word) >= lenWord:
            setWords.add(word)
    return (idDoc, list(setWords))

In [11]:
extractWords = re.map(getFeatures).filter(lambda x: x!=None)
extractWords.persist(storageLevel=StorageLevel(True, True, False, False))

PythonRDD[4] at RDD at PythonRDD.scala:53

In [12]:
vocabulary = extractWords.flatMap(lambda x: x[1])

In [13]:
for word in vocabulary.collect()[0:5]:
    print(word)
    # print('\n\n\n')

fundamentalist
BELIEF
Chicago
share
argument
accept
middle
embrace
something
atheism
provide
PFirst
parks
Covalt
reject
party
forcing
evidece
after
christians
mention
considered
atheist
proud
American
States
concrete
place
Trust
dedicated
backing
Brian
bevans
religious
check
group
built
carina
addition
hospitals
hospital
there
Introduction
correctly
Might
someone
Hopkins
Atheism
services
haven
regarding
Evans
glory
Virgin
atheist
medical
Johns
atheism
provide
without
recall
making
interpretation
zillions
culture
Theists
another
apply
different
religion
authoritative
writes
there
question
militant
atheists
followers
commentary
moderate
TEKIG
Naren
available
posting
orthodox
deleted
differences
interpretations
standard
Stuff
newsgroups
article
translation
naren
concordance
resolve
disclaimers
considered
Hmmmmmmmmmmmm
Quran
WHich
opinion
Moslems
Bangladesh
deaths
Sudan
Riots
Excuses
apply
wonder
KILLINGS
Inquistions
writes
people
Standard
RELIGION
above
horrible
resulted
America
Naren
mil